# 2-1. 하스스톤 직업별 점유율 및 승률 통계 데이터 크롤링
<pre>
▶ 기간 : 2023년 11월 15일 ~ 11월 28일자 [최신 확장팩 "황야의 땅 결투" 출시일]
▶ 대상 : 전 서버에 존재하는 모든 등급 유저
▶ 사이트 : [하스스톤 메타](https://hsreplay.net/meta)
</pre>

## 라이브러리

In [1]:
import requests # HTTP 프로토콜을 이용하여 웹사이트로부터 데이터를 송수신 받기 위한 라이브러리
from bs4 import BeautifulSoup # 웹페이지의 HTML, XML 파일에서 데이터를 추출하는 라이브러리
import openpyxl # 엑셀파일 작업을 위한 라이브러리
import pandas as pd # 구조화된 데이터나 표 형식의 데이터 분석 및 조작을 위한 라이브러리

# 웹페이지를 자동으로 조작하기 위한 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager # 웹브라우저의 드라이버를 자동으로 관리하는 라이브러리

# Selenium에서 사용되는 웹요소를 찾기 위한 기능을 제공해주는 라이브러리
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pyperclip # 자동 로그인을 위한 변수 복사 라이브러리
import re  #  정규표현식을 사용해 숫자를 추출하기 위한 라이브러리
from time import sleep  # 실행을 일시적으로 멈춰 로딩을 기다리기 위한 라이브러리

# 설정을 완료했다는 출력
print("Setup Complete")

Setup Complete


## 함수

### 1) 엑셀 및 csv 함수

#### 1-1) create_excel(time, server, rank)
<pre>
엑셀 생성 및 저장하는 함수
직업별 점유율 시트 생성
</pre>

In [2]:
def create_excel(time, server, rank) : 
  # 엑셀 만들기
  wb = openpyxl.Workbook()

  # 워크시트 만들기
  ws = wb.active
  ws.title = " 직업별 점유율"

  # 데이터 추가하기
  ws.append([f"기간 : {time}, 대상 : {server} 서버의 {rank} 등급 유저"])
  ws.append(['직업', '덱유형', '승률', '점유율', '게임수'])

  # 엑셀 저장하기
  wb.save('./hs_meta.xlsx')

# 설정을 완료했다는 출력
print("Setup Complete")

Setup Complete


#### 1-2) create_sheet(wb, time, server, rank)
<pre>
엑셀 워크시트 생성 및 저장하는 함수
매치업(하스스톤 승률 통계) 시트 생성
</pre>

In [12]:
def create_sheet(wb, time, server, rank):
  # 매치업 시트 정의
  new_sheet = "매치업"

  # 기존 매치업 시트가 존재하면 삭제
  if new_sheet in wb.sheetnames:
    wb.remove(wb[new_sheet])

  # 워크시트 만들기
  ws = wb.create_sheet(title=new_sheet)
  
  # 데이터 추가하기
  ws.append([f"기간 : {time}, 대상 : {server} 서버의 {rank} 등급 유저"])

  # 엑셀 저장하기
  wb.save('./hs_meta.xlsx')
  
  # 값 반환
  return ws

# 설정을 완료했다는 출력
print("Setup Complete")

Setup Complete


#### 1-3) save_csv()
<pre>
매치업 시트를 .csv 파일로 저장하는 함수
UTF-8 인코딩을 사용하고, 파일 맨 앞에 BOM을 추가하여 한글 깨지지 않도록 하는 옵션
</pre>

In [13]:
def save_csv():
  # 엑셀 파일 직접 읽기
  # 경로, 시트 지정, 무시할 행, 인덱스 열
  df_matchups = pd.read_excel('./hs_meta.xlsx', sheet_name="매치업", skiprows=1, index_col=0)

  # CSV 파일로 저장
  df_matchups.to_csv('./hs_matchups.csv', encoding='utf-8-sig')
  
# 설정을 완료했다는 출력
print("Setup Complete")

Setup Complete


### 2) 드라이버 진입 및 관련 함수

#### 2-1) get_user_input()
<pre>
데이터를 추출할 기간과 대상을 입력받아 딕셔너리 값을 반환하는 함수
</pre>

In [14]:
def get_user_input():
  # 입력값 받기
  time = int(input("검색할 기간을 입력하세요.\n[최근7일 : 7 // 최근 3일 : 3 // 최근 1일 : 1 // 최신 패치 : 0]"))
  rank = input("검색할 랭크를 입력하세요.\n[전구간 : A // 브론즈 : B // 실버 : S // 골드 : G // 플래티넘 : P // 다이아 : D // 전설 : L // 상위1000 : 1000]")
  server = input("검색할 서버를 입력하세요.\n[전서버 : A // 아시아 : AP // 유럽 : EU // 북미 : US]")

  # 검색 범위 딕셔너리 정의
  time_dict = {7: "LAST_7_DAYS", 3: "LAST_3_DAYS", 1: "LAST_1_DAY", 0: "LAST_PATCH"}
  rank_dict = {"A": "ALL", "B": "BRONZE", "S": "SILVER", "G": "GOLD", "P": "PLATINUM", "D": "DIAMOND", "L": "LEGEND", "1000": "TOP_1000_LEGEND"}
  server_dict = {"A": "ALL", "AP": "REGION_KR", "EU": "REGION_EU", "US": "REGION_US"}

  # 입력값이 딕셔너리에 정의된 값들 중 하나에 속하지 않을 경우 예외처리
  if time not in time_dict or rank not in rank_dict or server not in server_dict:
      raise ValueError("올바르지 않은 입력값입니다.")

  # 딕셔너리 값 반환
  return time_dict[time], rank_dict[rank], server_dict[server]

# 설정을 완료했다는 출력
print("Setup Complete")

Setup Complete


#### 2-2) wait_for_element(driver, by, value, timeout=10)
<pre>
WebDriverWait을 사용하여 특정 웹 엘리먼트가 나타날 때까지 대기하는 함수
대기 시간은 10초로 설정
</pre>

In [15]:
def wait_for_element(driver, by, value, timeout=10):
  # 웹페이지에서 특정 엘리먼트가 나타날 때까지 대기
  try:
    element_present = EC.presence_of_element_located((by, value))  # 엘리먼트 찾는 방법과 값
    WebDriverWait(driver, timeout).until(element_present) # timeout 시간 동안 대기 
  # 예외 처리    
  except Exception as e:
    print(f"엘리먼트 대기 중 예외 발생: {e}")
    driver.quit() # 드라이버 종료
    
# 설정을 완료했다는 출력
print("Setup Complete")

Setup Complete


#### 2-3) driver_in()
<pre>
크롬 드라이버에 진입하고 웹자동화로 로그인을 하여 드라이버와 크롬 설정을 반환하는 함수
</pre>

In [16]:
def driver_in(): 

  # 크롬 설정
  chrome_options = Options() # 크롬의 설정을 조작하기 위한 객체 생성
  chrome_options.add_experimental_option("detach", True) # 크롬을 실행 후, 스크립트가 종료되더라도 브라우저가 계속 실행되도록 설정
  chrome_options.add_argument('--no-sandbox') # 브라우저 보안으로 인해 샌드박스 모드에서 실행되지 않도록 설정
  chrome_options.add_argument('--disable-dev-shm-usage') # /dev/shm - 공유메모리 사용, /dev/shm을 사용하지 않도록 설정해서 메모리 부족을 회피
  chrome_options.add_argument("--incognito") # 브라우저를 시크릿 모드로 실행

  # ChromeDriverManager를 사용하여 크롬 드라이버를 초기화
  driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()), options=chrome_options)

  # URL 열기
  url = "https://hsreplay.net/account/login"
  driver.get(url)
  
  # 엘리먼트가 나타날 때까지 대기
  wait_for_element(driver, By.CLASS_NAME, "promo-button")

  # 로그인 진입 버튼 클릭 
  btn = driver.find_element(By.CLASS_NAME,"promo-button")
  btn.click()

  # 엘리먼트가 나타날 때까지 대기
  wait_for_element(driver, By.CLASS_NAME, "input-block")

  # ID 입력
  id = driver.find_elements(By.CLASS_NAME, "input-block")[0]
  pyperclip.copy("tlthr763@naver.com") # ID 복사
  id.send_keys(Keys.CONTROL, 'v') # ID 붙여넣기

  # PW 입력
  pw = driver.find_elements(By.CLASS_NAME, "input-block")[1]
  pyperclip.copy("qwer123657.") # 패스워드 복사
  pw.send_keys(Keys.CONTROL, 'v') # 패스워드 붙여넣기

  # 로그인 버튼 클릭
  log_btn = driver.find_element(By.CLASS_NAME, "submit-button")
  log_btn.click()

  # 드라이버, 크롬설정 반환
  return driver, chrome_options

# 설정을 완료했다는 출력
print("Setup Complete")

Setup Complete


#### 2-4) open_url(time, rank, server)
<pre>
입력받은 기간과 대상에 따라서 다른 URL을 반환하는 함수
</pre>

In [17]:
def open_url(time, rank, server):
  # time, rank, server에 따라 다른 URL을 생성하는 함수
  if time == "LAST_PATCH":
      # time이 "LAST_PATCH"인 경우
      url = f"https://hsreplay.net/meta/#tab=archetypes"
  elif rank == "ALL" and server == "ALL":
      # rank가 "ALL"이고 server가 "ALL"인 경우
      url = f"https://hsreplay.net/meta/#tab=archetypes&timeFrame={time}"
  elif rank == "ALL":
      # rank가 "ALL"인 경우
      url = f"https://hsreplay.net/meta/#tab=archetypes&timeFrame={time}&region={server}"
  elif server == "ALL":
      # server가 "ALL"인 경우
      url = f"https://hsreplay.net/meta/#tab=archetypes&timeFrame={time}&rankRange={rank}"
  else:
      # 그 외, time, rank, server가 특정 대상인 경우
      url = f"https://hsreplay.net/meta/#tab=archetypes&timeFrame={time}&rankRange={rank}&region={server}"
  
  # URL 반환
  return url

# 설정을 완료했다는 출력
print("Setup Complete")

Setup Complete


### 3) 데이터 추출 함수

#### 3-1) get_popular_info(driver,data)
<pre>
직업 점유율 데이터를 추출하는 함수
</pre>

In [18]:
def get_popular_info(driver,data):
  # 직업 클래스 선택
  hero_elements = driver.find_elements(By.CSS_SELECTOR, '.class-box')

  # 직업별 정보 저장 루프
  for element in hero_elements:
      # 직업 이름 추출
      hero = element.find_element(By.CSS_SELECTOR, '.box-title').text
      # 해당 직업에 대한 덱 유형과 정보 클래스
      deck_elements = element.find_elements(By.CSS_SELECTOR, '.table-row-header')
      stats_elements = element.find_elements(By.CSS_SELECTOR, '.table-cell')

      # 덱 유형별 이름과 정보 저장 루프
      for deck_element in deck_elements:
        # 덱 이름 추출
        type_name = deck_element.find_element(By.CSS_SELECTOR, '.tooltip-wrapper').text

        # 승률, 점유율, 게임 수 추출
        win_rate = stats_elements[0].text
        popularity = stats_elements[1].text
        games = stats_elements[2].text
      
        # 데이터를 리스트에 추가
        data.append([hero, type_name, win_rate, popularity, games])

        # 덱 정보를 담고 있는 stats_elements에서 이미 처리한 스텟 정보를 제거
        # 슬라이싱을 사용하여 리스트 앞쪽에서 3개의 원소를 제거
        # 다시 루프를 통해 다음 덱 정보를 추출할 때 업데이트된 리스트 사용
        # 이를 통해 다음 덱의 정보를 올바르게 추출이 가능하다
        stats_elements = stats_elements[3:]
        
# 설정을 완료했다는 출력
print("Setup Complete")

Setup Complete


#### 3-2) get_deck(driver,data)
<pre>
덱별 이름을 추출하는 함수
</pre>

In [19]:
def get_deck(driver,data):
  # 덱 정보 클래스 선택
  arche_elements = driver.find_elements(By.CLASS_NAME, 'matchup-row-header')

  # 덱별 이름 저장 루프
  for arche in arche_elements:
      # 현재 행에서 덱 이름 추출
      try :
        is_arche=arche.find_element(By.CLASS_NAME, 'archetype-name') # 덱 이름 클래스 선택
        deck_type = is_arche.text # 텍스트 추출
        data.append(deck_type) # 데이터를 data 리스트에 추가
      # 예외 처리 
      except:
        not_arche=arche.find_element(By.TAG_NAME, 'aside') # 기타 클래스 선택
        deck_type = not_arche.text # 텍스트 추출
        data.append(deck_type) # 데이터를 리스트에 추가
        
# 설정을 완료했다는 출력
print("Setup Complete")

Setup Complete


#### 3-3) get_deck_matrix(driver, data, ws)
<pre>
덱별 승률 통계 데이터를 추출하는 함수
</pre>

In [20]:
def get_deck_matrix(driver, data, ws):
    # 덱별 승률 통계 요소 선택
    rate_matrix = driver.find_elements(By.CLASS_NAME, 'matchup-cell')

    # 데이터를 엑셀에 추가할 행 인덱스 (B3 셀)
    row_index = 3

    # 덱별 승률 통계 저장 루프
    for element in rate_matrix:
        # 덱별 승률 추출
        rate = element.find_element(By.CLASS_NAME, 'tooltip-wrapper').text

        # rate를 적절히 처리하여 데이터 추출
        # rate를 콤마(,)로 분리하여 리스트로 변환
        cell_data = rate.split(',')

        # 데이터를 리스트에 추가
        data.extend(cell_data)

        # 14개의 데이터가 모일 때마다 엑셀에 행 추가
        if len(data) == 14:
            # 행, 열 수에 맞게 엑셀에 추가
            # col_index - 열 인덱스, value - data 리스트에서 가져온 값
            # enumerate : data 리스트의 각 값과 해당하는 인덱스를 반환
            for col_index, value in enumerate(data, start=2): # 덱 이름이 첫번째 열이므로, 열 인덱스를 2부터 시작 
                ws.cell(row=row_index, column=col_index, value=value) # 시트에서 지정한 행과 열에 값 추가

            data.clear()  # 다음 승률을 추출하기 위해서 리스트 비우기
            row_index += 1  # 행 인덱스 증가

        # 14개의 데이터를 처리한 경우에 루프 종료
        if row_index > 16:  # 14개의 행을 처리했으므로 종료
            break
        
# 설정을 완료했다는 출력
print("Setup Complete")


Setup Complete


## 실행 코드

In [24]:
# 입력값 함수
time, rank, server = get_user_input()

# 엑셀 생성 함수
create_excel(time, rank, server)

# 웹브라우저 진입 함수
driver, chrome_options = driver_in()
sleep(2) # 로그인 후 2초동안 대기

# URL 링크 여는 함수
url = open_url(time, rank, server)
driver.get(url)

# 엘리먼트가 나타날 때까지 대기
wait_for_element(driver, By.CLASS_NAME, 'class-box-container')

# 엑셀 열기
wb = openpyxl.load_workbook('./hs_meta.xlsx')
ws = wb.active

# 점유율 데이터를 저장할 리스트 생성
data = []

# 점유율 데이터 추출 함수
get_popular_info(driver,data)

# .xlsx 형식으로 엑셀에 추가
for row in data: # 점유율 데이터를 엑셀에 추가
    ws.append(row)

# .xlsx 형식으로 엑셀 저장
wb.save('./hs_meta.xlsx')

# 창의 너비와 높이 설정 
driver.set_window_size(1592, 1039)

# '매치업' 탭 클릭
matchup_tab = driver.find_element(By.ID, 'tab-matchups')
matchup_tab.click()

# 엑셀 새로운 시트 생성
ws_match = create_sheet(wb, time, rank, server)

# 덱 이름을 저장할 리스트 생성
data2 = ["",]

# 덱 이름 추출 함수
get_deck(driver,data2)
            
# .xlsx 형식으로 엑셀에 추가
ws_match.append(data2) # 덱 이름 리스트를 열 이름으로 추가
data2.pop(0) # 공백 제거

# 덱 이름 리스트를 인덱스로 추가
for row in data2:
    ws_match.append([row])

# 승률 데이터를 저장할 리스트 생성
data3 = []

# 승률 데이터 추출 함수
get_deck_matrix(driver, data3, ws_match)

# .xlsx 형식으로 엑셀 저장
wb.save('./hs_meta.xlsx')

# 엑셀 종료
wb.close()

# 크롬창 종료
driver.close()

# 데이터를 포함한 리스트에서 DataFrame을 생성
df_popular = pd.DataFrame(data, columns=['Hero', 'Deck', 'Winrate', 'Popular', 'Games'])

# DataFrame을 UTF-8-sig 인코딩으로 CSV 파일로 저장
df_popular.to_csv('./hs_meta.csv', encoding='utf-8-sig', index=False)

# Matchups 시트를 UTF-8-sig 인코딩으로 CSV 파일로 저장
save_csv()

# 실행을 완료했다는 출력
print("Data Crawling Complete")

Data Crawling Complete
